In [ ]:
# Import necessary libraries for data processing, model building, and evaluation
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from scipy import stats
from collections import Counter
from scipy.stats import pointbiserialr
from math import sqrt
import copy
import math
from sklearn.utils import resample
import torch
from scipy.stats import chi2
from torch_geometric.data import Dataset, Data
from sklearn import preprocessing
from scipy.sparse import lil_matrix
from scipy.sparse import coo_matrix
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import ParameterGrid

import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

def data_get():
    disese = pd.read_csv(r"D:\jupyter\New\dateset\mRNA_LncRNA_Non_Small_Cell_Lung_Carcinoma.csv")
    health = pd.read_csv(r"D:\jupyter\New\dateset\mRNA_LncRNA_Non_Small_Cell_Lung_Carcinoma_HC.csv")
    
    # 提取“蛋白编码”基因的数据
    dat = disese[disese['gene_type'] == "protein_coding"]
    tes = health[health['gene_type'] == "protein_coding"]
    column_name = dat.iloc[:, 2].values  # 获取基因名称
    # 获取样本的数量：数据的列是样本，行是基因
    dat_samples = dat.iloc[:, 6:].T  # 只取样本数据，转置后每行是一个样本
    tes_samples = tes.iloc[:, 6:].T  # 同样对健康数据处理，转置后每行是一个样本
    
    return dat_samples, tes_samples, column_name

# 使用修改后的数据加载函数
dat_ace, tes_ace, column_name = data_get()

# 合并数据
df_data = pd.concat([dat_ace, tes_ace], axis=0)  # 沿着行（样本）合并数据

# 假设标签在第二列
df_data.columns = [column_name]

# 标签处理：疾病为1，健康为0
dis_lab = [1] * len(dat_ace)  # 疾病标签
HC_lab = [0] * len(tes_ace)  # 健康标签
label = dis_lab + HC_lab  # 合并标签

df_Z = scale(df_data, axis=0, with_mean=True, with_std=True, copy=True)
expr_data = pd.DataFrame(df_Z, columns=df_data.columns.get_level_values(0).values)


In [ ]:
data = pd.read_csv(r'D:\jupyter\new_funtion\data\data1\o_adata.csv')
df_data = data.drop(columns='Label')
label = data['Label']
df_Z = preprocessing.scale(df_data, axis=0, with_mean=True, with_std=True, copy=True)
expr_data = pd.DataFrame(df_Z,columns=df_data.columns.get_level_values(0).values, index=df_data.index)

In [ ]:
# 加载基因间关系数据
interaction_data_path = r'D:\jupyter\new_funtion\2024_7_9\download.txt'
interaction_data = pd.read_csv(interaction_data_path, sep="\t", header=None)
interaction_data.columns = ['raid', 'actor1', 'cat1', 'sp1', 'actor2', 'cat2', 'sp2', 'score']

# 准备基因交互网络（图）
genes = expr_data.columns  # 基因列
adj_matrix = lil_matrix((len(genes), len(genes)))  # 创建空的邻接矩阵

# 创建基因到索引的映射
gene_to_idx = {gene: idx for idx, gene in enumerate(genes)}

# 填充邻接矩阵
for index, row in interaction_data.iterrows():
    if row['actor1'] in gene_to_idx and row['actor2'] in gene_to_idx:
        i, j = gene_to_idx[row['actor1']], gene_to_idx[row['actor2']]
        adj_matrix[i, j] = 1
        adj_matrix[j, i] = 1

# 转换为CSR格式以提高计算效率
adj_matrix_csr = adj_matrix.tocsr()
adj_matrix_coo = coo_matrix(adj_matrix_csr)  # 转换为COO格式用于边的索引

# 准备图神经网络的边索引
edge_index = torch.tensor([adj_matrix_coo.row, adj_matrix_coo.col], dtype=torch.long)

# 准备特征矩阵（基因表达数据）
x = torch.tensor(expr_data.values.T, dtype=torch.float)  # 特征应该对应到节点
data = Data(x=x, edge_index=edge_index)


In [ ]:
# Model Definition:
# `ImprovedGCN` is a Graph Convolutional Network model with multiple convolution layers.
# The model learns both node features and topological graph structure through the graph convolution layers.
# It includes dropout and batch normalization to improve performance.
# 加载基因间关系数据
interaction_data_path = r'D:\jupyter\new_funtion\2024_7_9\download.txt'
interaction_data = pd.read_csv(interaction_data_path, sep="\t", header=None)
interaction_data.columns = ['raid', 'actor1', 'cat1', 'sp1', 'actor2', 'cat2', 'sp2', 'score']

# 准备基因交互网络（图）
genes = expr_data.columns  # 基因列
adj_matrix = lil_matrix((len(genes), len(genes)))  # 创建空的邻接矩阵

# 创建基因到索引的映射
gene_to_idx = {gene: idx for idx, gene in enumerate(genes)}

# 填充邻接矩阵
for index, row in interaction_data.iterrows():
    if row['actor1'] in gene_to_idx and row['actor2'] in gene_to_idx:
        i, j = gene_to_idx[row['actor1']], gene_to_idx[row['actor2']]
        adj_matrix[i, j] = 1
        adj_matrix[j, i] = 1

# 转换为CSR格式以提高计算效率
adj_matrix_csr = adj_matrix.tocsr()
adj_matrix_coo = coo_matrix(adj_matrix_csr)  # 转换为COO格式用于边的索引

# 准备图神经网络的边索引
edge_index = torch.tensor([adj_matrix_coo.row, adj_matrix_coo.col], dtype=torch.long)

# 准备特征矩阵（基因表达数据）
x = torch.tensor(expr_data.values.T, dtype=torch.float)  # 特征应该对应到节点
data = Data(x=x, edge_index=edge_index)
features = torch.tensor(expr_data.values.T, dtype=torch.float)  # 注意转置以便每列特征对应一个节点
print("---------------------------------------------------------------")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ImprovedGCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels=64):
        super(ImprovedGCN, self).__init__()
        # 添加多层GCN和残差连接
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_node_features)
        
        # 添加批归一化
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_channels)
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_channels)
        
        # 添加dropout防止过拟合
        self.dropout = torch.nn.Dropout(0.2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # 第一层
        identity = x
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第二层
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第三层
        x = self.conv3(x, edge_index)
        
        # 添加残差连接
        return x + identity

class ImprovedGraphLoss(nn.Module):
    def __init__(self, alpha=0.1):
        super(ImprovedGraphLoss, self).__init__()
        self.alpha = alpha
        
    def forward(self, predictions, targets, edge_index):
        # MSE损失
        mse_loss = F.mse_loss(predictions, targets)
        
        # 添加拓扑结构保持损失
        row, col = edge_index
        edge_loss = torch.mean(torch.abs(
            torch.norm(predictions[row] - predictions[col], dim=1) -
            torch.norm(targets[row] - targets[col], dim=1)
        ))
        
        return mse_loss + self.alpha * edge_loss

# 模型初始化
input_dim = features.shape[1]
model = ImprovedGCN(input_dim)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
criterion = ImprovedGraphLoss()

# 训练过程
best_loss = float('inf')
best_model = None

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss = criterion(out, data.x, data.edge_index)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    # 学习率调整
    scheduler.step(loss)
    
    # 保存最佳模型
    if loss < best_loss:
        best_loss = loss
        best_model = copy.deepcopy(model)
        
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 使用最佳模型进行预测
model = best_model
out = model(data)

# 将模型输出转换为DataFrame格式
out_np = out.detach().cpu().numpy()
data_gcn = pd.DataFrame(out_np.T, columns=[f'Feature_{i}' for i in range(out_np.shape[0])], index=expr_data.index)
data_gcn.columns = expr_data.columns.get_level_values(0).values 

In [ ]:
# Model Definition:
# `ImprovedGCN` is a Graph Convolutional Network model with multiple convolution layers.
# The model learns both node features and topological graph structure through the graph convolution layers.
# It includes dropout and batch normalization to improve performance.
features = torch.tensor(expr_data.values.T, dtype=torch.float)  # 注意转置以便每列特征对应一个节点
print("---------------------------------------------------------------")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ImprovedGCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels=64):
        super(ImprovedGCN, self).__init__()
        # 添加多层GCN和残差连接
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_node_features)
        
        # 添加批归一化
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_channels)
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_channels)
        
        # 添加dropout防止过拟合
        self.dropout = torch.nn.Dropout(0.2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # 第一层
        identity = x
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第二层
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第三层
        x = self.conv3(x, edge_index)
        
        # 添加残差连接
        return x + identity

class ImprovedGraphLoss(nn.Module):
    def __init__(self, alpha=0.1):
        super(ImprovedGraphLoss, self).__init__()
        self.alpha = alpha
        
    def forward(self, predictions, targets, edge_index):
        # MSE损失
        mse_loss = F.mse_loss(predictions, targets)
        
        # 添加拓扑结构保持损失
        row, col = edge_index
        edge_loss = torch.mean(torch.abs(
            torch.norm(predictions[row] - predictions[col], dim=1) -
            torch.norm(targets[row] - targets[col], dim=1)
        ))
        
        return mse_loss + self.alpha * edge_loss

# 模型初始化
input_dim = features.shape[1]
model = ImprovedGCN(input_dim)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)
criterion = ImprovedGraphLoss()

# 训练过程
best_loss = float('inf')
best_model = None

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss = criterion(out, data.x, data.edge_index)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    
    # 学习率调整
    scheduler.step(loss)
    
    # 保存最佳模型
    if loss < best_loss:
        best_loss = loss
        best_model = copy.deepcopy(model)
        
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 使用最佳模型进行预测
model = best_model
out = model(data)

# 将模型输出转换为DataFrame格式
out_np = out.detach().cpu().numpy()
data_gcn = pd.DataFrame(out_np.T, columns=[f'Feature_{i}' for i in range(out_np.shape[0])], index=expr_data.index)
data_gcn.columns = expr_data.columns.get_level_values(0).values 

In [ ]:
# Import necessary libraries for data processing, model building, and evaluation
import pandas as pd
from scipy.stats import pearsonr

# 假设 data_gcn 和 expr_data 已经存在，且具有相同的形状和列名称（基因名）

# 删除重复列名（确保每列唯一）
data_gcn = data_gcn.loc[:, ~data_gcn.columns.duplicated(keep='first')]
expr_data = expr_data.loc[:, ~expr_data.columns.duplicated(keep='first')]

# 创建一个 DataFrame 来存储每个基因的皮尔森相关系数
pearson_scores = []

# 填充缺失值为 0（根据实际需求调整）
data_gcn = data_gcn.fillna(0)
expr_data = expr_data.fillna(0)

for gene in data_gcn.columns:
    try:
        # 提取当前基因的值（确保是 Series）
        gcn_values = data_gcn[gene].squeeze()
        original_values = expr_data[gene].squeeze()
        
        # 转换为数值型数据（处理可能的非数值类型）
        gcn_values = pd.to_numeric(gcn_values, errors='coerce')
        original_values = pd.to_numeric(original_values, errors='coerce')
        
        # 删除含有 NaN 的行
        valid_data = pd.DataFrame({
            'gcn': gcn_values,
            'original': original_values
        }).dropna()
        
        if valid_data.empty:
            raise ValueError(f"No valid data for gene {gene} after dropping NaN.")
        
        # 计算皮尔森相关系数
        correlation, _ = pearsonr(valid_data['gcn'], valid_data['original'])
        pearson_scores.append({"Gene": gene, "Pearson_Correlation": correlation})
    except Exception as e:
        print(f"Error processing gene {gene}: {e}")
        continue

# 转换为 DataFrame
pearson_df = pd.DataFrame(pearson_scores)

# 筛选出皮尔森系数在 -1 到 -0.8 或 0.8 到 1 之间的基因
filtered_genes = pearson_df[
    (pearson_df["Pearson_Correlation"] >= 0.7) | 
    (pearson_df["Pearson_Correlation"] <= -0.7)
]

# 提取符合条件的基因列表
selected_genes = filtered_genes["Gene"].tolist()

# 从 data_gcn 和 expr_data 中提取对应基因的数据
selected_data_gcn = data_gcn[selected_genes]
selected_expr_data = expr_data[selected_genes]

# 打印结果
print("Selected Genes and Pearson Correlation Coefficients:")
print(pearson_df)
print("\nData from data_gcn:")
print(selected_data_gcn.head())
print("\nData from expr_data:")
print(selected_expr_data.head())

In [ ]:
# Import necessary libraries for data processing, model building, and evaluation
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error  # Import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore")

# 累计ACE的总和
total_ace = 0  
# 用于存储ACE值
ace_values = []

# 假设expr_data, data_gcn已经定义且格式正确
# expr_data: 基因表达数据
# data_gcn: GCN模型输出数据
# feature_columns = expr_data.columns.get_level_values(0).values 
feature_columns = filtered_genes["Gene"]  # 替换为正确的列名

# 遍历每一个特征列
for column_name in feature_columns:
    try:
        # 确保列存在
        if column_name not in data_gcn or column_name not in expr_data:
            print(f"Column {column_name} not found in one of the datasets.")
            continue

        f_s_array = data_gcn.drop(columns=[column_name]).sum(axis=1)
        
        f = data_gcn[column_name]
        
        # 计算差异和方差
        diff = f - f_s_array
        alpha = 0.1   # 也可根据你的需求修改
        c = 0.0       # 偏置项

# 直接用 tanh 函数
        e_f = np.tanh(alpha * diff + c)
        
        # 准备数据
        ax_0 = [1] * expr_data.shape[0]
        Logist_data = pd.DataFrame({
            'a0': ax_0,
            'f': f,
            'f^2': f ** 2,
            'e(f)': e_f,
            'e(f)^2': e_f ** 2,
            'f*e(f)': f * e_f
        }, index=expr_data.index)
        
        # 标准化数据
        scaler = StandardScaler()
        Logist_data = pd.DataFrame(
            scaler.fit_transform(Logist_data),
            columns=Logist_data.columns,
            index=Logist_data.index
        )

        y = label  # 假设label已定义，标签数据
        X_train, X_test, y_train, y_test = train_test_split(Logist_data, y, test_size=0.2, random_state=0)

        # 使用交叉验证评估模型
        model_logi = LogisticRegression(max_iter=10000)  # 增加max_iter防止不收敛
        cv_scores = cross_val_score(model_logi, X_train, y_train, cv=10, scoring='accuracy')

        print(f"Cross-validation scores for {column_name}: {cv_scores}")
        print(f"Mean cross-validation score: {np.mean(cv_scores)}")

        # 训练模型
        model_logi.fit(X_train, y_train)
        y_prob = model_logi.predict_proba(X_test)[:, 1]  # 获取阳性类别的概率

        # 计算ACE（均方误差）
        ace = mean_squared_error(y_test, y_prob)
        
        ace_values.append({"特征": column_name, "ACE": ace})
        print(f"特征 '{column_name}' 的 ACE: {ace}")

    except Exception as e:
        print(f"Error processing column {column_name}: {e}")
        continue

# 转换ACE值为DataFrame
ace_df = pd.DataFrame(ace_values)
print(ace_df)


In [ ]:
ace_df_sorted = ace_df.sort_values(by="ACE", ascending=True)
ace_df_sorted
# 将 ACE 值列表转换为 DataFrame
ace_df_sorted_csv = pd.DataFrame(ace_df_sorted)

# 保存 ACE 值 DataFrame 为 CSV 文件
ace_csv_sortd_file = r'D:\jupyter\New\result\o_adata_tanh_ace_sorted_result.csv'
ace_df_sorted_csv.to_csv(ace_csv_sortd_file, index=False)

In [ ]:
# Import necessary libraries for data processing, model building, and evaluation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

def plot_confusion_matrices(y_true, all_predictions, filename):
    plt.figure(figsize=(20, 15))
    
    classes = np.unique(y_true)
    
    for idx, (clf_name, predictions) in enumerate(all_predictions.items(), 1):
        plt.subplot(2, 2, idx)
        
        # 确保预测值和真实值的顺序一致
        cm = confusion_matrix(y_true, predictions, labels=classes)
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=classes, 
                    yticklabels=classes)
        plt.title(f"{clf_name} Confusion Matrix")
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
    
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

def plot_roc_curves(y_true, classifiers_probas, filename):
    plt.figure(figsize=(10, 8))
    
    for clf_name, probas in classifiers_probas.items():
        if probas is not None:
            fpr, tpr, _ = roc_curve(y_true, probas)
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{clf_name} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curves')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

class FixedStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        return self.scaler.transform(X)

def evaluate_classifier(classifier, data, label, features, max_features):
    results = []
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    for num_features in range(1, max_features + 1):
        selected_features = features[:num_features]
        X = data[selected_features]
        y = np.array(label)

        fold_accuracy = []
        fold_f1 = []
        fold_auc = []

        for train_index, test_index in cv.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]

            pipeline = Pipeline([
                ('scaler', FixedStandardScaler()),
                ('classifier', clone(classifier))
            ])

            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)

            fold_accuracy.append(accuracy_score(y_test, y_pred))
            fold_f1.append(f1_score(y_test, y_pred, average='weighted'))

            if hasattr(classifier, 'predict_proba'):
                y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
                try:
                    fold_auc.append(roc_auc_score(y_test, y_pred_proba))
                except:
                    fold_auc.append(None)
            else:
                fold_auc.append(None)

        results.append({
            'NumFeatures': num_features, 
            'Accuracy': np.mean(fold_accuracy), 
            'F1Score': np.mean(fold_f1),
            'AUC': np.mean([auc for auc in fold_auc if auc is not None]) if fold_auc else None
        })
    
    return pd.DataFrame(results)

def data_get_corss():
    disese = pd.read_csv(r"D:\jupyter\New\dateset\mRNA_LncRNA_Multiple_Sclerose.csv")
    health = pd.read_csv(r"D:\jupyter\New\dateset\mRNA_LncRNA_Multiple_Sclerose_HC.csv")
    dat = disese
    tes = health
    return dat, tes

def merge_data(dat, tes, r_res):
    data = pd.merge(dat, tes)
    df = data.iloc[:, 6:]
    Y = list(r_res.keys())
    df = df.transpose()
    y = data.iloc[:, 2].values
    df.columns = [y]
    df = df.loc[:, Y]
    return df

def main():
    #dat_ace, tes_ace = data_get_corss()
    #data_merge = pd.merge(dat_ace, tes_ace)
    #df_data = data_merge.iloc[:, 6:]
    #df_data = df_data.transpose()
    #dis_lab = [1]*dat_ace.iloc[0:,6:].shape[1]
    #HC_lab = [0]*tes_ace.iloc[0:,6:].shape[1]
    #df_lab = dis_lab+HC_lab

    data = pd.read_csv(r'D:\jupyter\new_funtion\data\data1\o_adata.csv')
    df_data = data.drop(columns='Label')
    df_lab = data['Label']

    ace_df_sorted_csv = pd.read_csv(r"D:\jupyter\New\result\o_adata_tanh_ace_sorted_result.csv")
    features = ace_df_sorted_csv['特征']
    #y = dat_ace.iloc[:, 2].values
    #df_data.columns = [y]
    data_test = df_data
    label = df_lab
    label_array = np.array(label)

    classifiers = {
        'SVC': SVC(probability=True),
        'XGBClassifier': XGBClassifier(),
        'GaussianNB': GaussianNB(),
        'DecisionTree': DecisionTreeClassifier()
    }

    feature_ranges = [50, 100, 200]

    for max_features in feature_ranges:
        print(f"\nEvaluating with maximum {max_features} features...")
        
        results_summary = []
        final_predictions = {}
        final_probabilities = {}

        for clf_name, clf in classifiers.items():
            print(f"Evaluating {clf_name}...")
            result_df = evaluate_classifier(clf, data_test, label_array, features, max_features)

            result_df_cleaned = result_df.dropna(subset=['F1Score'])
            max_f1_index = result_df_cleaned['F1Score'].idxmax()
            best_num_features = int(result_df_cleaned.loc[max_f1_index]['NumFeatures'])
            best_f1_score = result_df_cleaned.loc[max_f1_index]['F1Score']
            best_accuracy = result_df_cleaned.loc[max_f1_index]['Accuracy']
            best_auc = result_df_cleaned.loc[max_f1_index]['AUC']

            print(f"{clf_name} - Best number of features: {best_num_features}, Best F1 Score: {best_f1_score}")

            X_selected = data_test[features[:best_num_features]]
            cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            
            # 用于存储每个样本的预测结果和概率
            predictions = np.zeros(len(label_array))
            probabilities = np.zeros(len(label_array))
            
            # 记录每个样本被预测的次数
            prediction_counts = np.zeros(len(label_array))

            for train_index, test_index in cv.split(X_selected, label_array):
                X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
                y_train = label_array[train_index]

                pipeline = Pipeline([
                    ('scaler', FixedStandardScaler()),
                    ('classifier', clone(clf))
                ])

                pipeline.fit(X_train, y_train)
                fold_predictions = pipeline.predict(X_test)
                
                # 累加预测结果
                predictions[test_index] += fold_predictions
                prediction_counts[test_index] += 1
                
                if hasattr(clf, 'predict_proba'):
                    fold_probas = pipeline.predict_proba(X_test)[:, 1]
                    probabilities[test_index] += fold_probas

            # 计算最终预测结果（多数投票）
            predictions = predictions / prediction_counts
            predictions = (predictions > 0.5).astype(int)
            
            # 计算最终概率（平均）
            if hasattr(clf, 'predict_proba'):
                probabilities = probabilities / prediction_counts
            
            final_predictions[clf_name] = predictions
            final_probabilities[clf_name] = probabilities

            results_summary.append({
                'Classifier': clf_name,
                'Best_Num_Features': best_num_features,
                'Best_F1_Score': best_f1_score,
                'Best_Accuracy': best_accuracy,
                'Best_AUC': best_auc
            })

        # 保存结果
        confusion_matrix_path = f'D:\\jupyter\\New\\result\\cm\\{max_features}-o_adata_tanh_confusion_matrices.png'
        roc_curve_path = f'D:\\jupyter\\New\\result\\roc\\{max_features}-o_adata_tanh_roc_curves.png'
        
        plot_confusion_matrices(label_array, final_predictions, confusion_matrix_path)
        plot_roc_curves(label_array, final_probabilities, roc_curve_path)

        best_result_df = pd.DataFrame(results_summary)
        result_csv_path = f'D:\\jupyter\\New\\result\\f1-score\\{max_features}-o_adata_tanh_best_classification_results.csv'
        best_result_df.to_csv(result_csv_path, index=False)

if __name__ == "__main__":
    main()